In [2]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


#Aggregating The Features into one sigle column using VectorAssembler

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [15]:
output = assembler.transform(data) #creating a new dataframe with the aggregated feature column from the exisiting one by using VectorAssembler object's transfrom method

In [16]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [17]:
final_data = output.select('features','Yearly Amount Spent')

In [18]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

#Split in Train Test Data

In [19]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean|  500.4294646325872|
| stddev|   77.4081726633345|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean| 496.58316679122106|
| stddev|  84.01498252131155|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



#Creating a Linear Regression Model

In [22]:
lr = LinearRegression(labelCol='Yearly Amount Spent') #mentioning label col as it has a different name

#Training the model based on the train dataset

In [23]:
lr_Model = lr.fit(train_data) 

#Evalutaing the Model with Test Data

In [25]:
test_results = lr_Model.evaluate(test_data)

In [27]:
test_results.residuals.show() #Residuals in a statistical or machine learning model are the differences between observed and predicted values of data

+-------------------+
|          residuals|
+-------------------+
|  11.19255549520426|
| -4.278103836492164|
|-3.6905731646024833|
| -6.723519185537839|
| 16.639763597735566|
| -6.351463077577989|
| -10.76885314340484|
| -4.022292591930409|
|-18.035521397340347|
|  7.888896708433265|
| -9.246661346365215|
|  8.000146859325525|
|-10.772761324030967|
| -5.463347301650572|
| -8.870080401599068|
| 12.138631030031206|
| -3.479321814072023|
| 0.5617493877511492|
|   17.8483242640159|
| 0.4098897974954525|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

9.294180783540021

In [29]:
test_results.r2

0.9876770621259687

In [30]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



#Deploying The Model to Predict On Unlabeled Data

In [32]:
unlabeled_data = test_data.select('features')

In [34]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[31.2681042107507...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.6098395733896...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8648325480987...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9096268275227...|
|[31.9673209478824...|
|[32.0047530203648...|
|[32.0180740106320...|
|[32.0444861274404...|
+--------------------+
only showing top 20 rows



In [36]:
predictions = lr_Model.transform(unlabeled_data)

In [41]:
predictions.show()
test_data.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 397.4477955774232|
|[31.2681042107507...| 427.7486370103161|
|[31.4252268808548...| 534.4572918193644|
|[31.4474464941278...|425.32626128076186|
|[31.6098395733896...| 427.9057860533726|
|[31.7242025238451...| 509.7393503655385|
|[31.8093003166791...|  547.540752506246|
|[31.8124825597242...|396.83263757572763|
|[31.8164283341993...| 519.1580129009967|
|[31.8209982016720...| 416.7863843047801|
|[31.8279790554652...|449.24940889330674|
|[31.8512531286083...|464.99209980747287|
|[31.8648325480987...|450.66404180084464|
|[31.8745516945853...|397.74859154791807|
|[31.8854062999117...| 398.9733533740746|
|[31.9096268275227...|  551.307404643208|
|[31.9673209478824...| 449.2291630537243|
|[32.0047530203648...| 463.1842317328783|
|[32.0180740106320...| 339.9347864812994|
|[32.0444861274404...| 447.8199393890545|
+--------------------+------------